In [ ]:
%reload_ext eradiate
%reload_ext eradiate.notebook.tutorials

import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

import eradiate
from eradiate import unit_registry as ureg

eradiate.set_mode("mono")
eradiate.config.progress = 1

In [ ]:
origin = [0, 0,eradiate.constants.EARTH_RADIUS.m_as(ureg.m)] * ureg.m
vertical = np.array([0, 0, 1])

def exp(sza):
    return eradiate.experiments.AtmosphereExperiment(
        geometry="spherical_shell",
        atmosphere={
            "type": "molecular",
            "construct": "ussa_1976",
        },
        illumination={
            "type": "directional",
            "zenith": sza,
            "azimuth": 180,
        },
        measures={
            "type": "perspective", 
            "origin": origin + [100, 0, 0] * ureg.km,
            "target": origin + [0, 0, 10] * ureg.km,
            "up": vertical,
            "film_resolution": (32, 16),
            "spectral_cfg": {"wavelengths": [440, 550, 660] * ureg.nm}
        }
    )

In [ ]:
result = []
for sza in np.arange(0, 96, 10) * ureg.deg:
    print(f"{sza = :~}")
    result.append(eradiate.run(exp(sza), spp=100).copy())

In [ ]:
ds = xr.combine_by_coords(result, combine_attrs="drop")
ds

In [ ]:
for sza in ds.sza.values:
    
    plt.imshow(
        eradiate.xarray.interp.dataarray_to_rgb(
            ds.radiance.sel(sza=sza),
            channels=[("w", 660), ("w", 550), ("w", 440)],
        ),
    )
    plt.axis("off")
    sza *= ureg.deg
    plt.title(f"{sza = :~}")
    plt.show()
    plt.close()

In [ ]:
origin = [0, 0, 0] * ureg.m

exp = eradiate.experiments.AtmosphereExperiment(
    atmosphere={
        "type": "molecular",
        "construct": "ussa_1976",
    },
    illumination={
        "type": "directional",
        "zenith": 85,
        "azimuth": 0,
    },
    measures={
        "type": "perspective", 
        "origin": origin + [10, 0, 0] * ureg.km,
        "target": origin + [0, 0, 1] * ureg.km,
        "up": [0, 0, 1],
        "film_resolution": (32, 16),
        "spectral_cfg": {"wavelengths": [440, 550, 660] * ureg.nm}
    }
)

result = eradiate.run(exp, spp=1000)
plt.imshow(
    eradiate.xarray.interp.dataarray_to_rgb(
        result.radiance,
        channels=[("w", 660), ("w", 550), ("w", 440)],
        normalize=True,
    ),
)